In [11]:
import pandas as pd
import os
import librosa
import numpy as np
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display

In [12]:
audio_dataset_path='C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/An-Nas/'
metadata=pd.read_csv('C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/An-Nas/metadata_114.csv')
metadata.head(20)

,slice_file_name,fold,classID,class
0,1_1.wav,1,114,An-Nas
1,1_2.wav,1,114,An-Nas
2,1_3.wav,1,114,An-Nas
3,1_4.wav,1,114,An-Nas
4,1_5.wav,1,114,An-Nas
5,1_6.wav,1,114,An-Nas
6,1.wav,2,114,An-Nas
7,2.wav,2,114,An-Nas
8,3.wav,2,114,An-Nas
9,4.wav,2,114,An-Nas


Feature Extraction

In [13]:
def mfcc_features_extractor_1sec(file):
    # audio, sample_rate = librosa.load(file_name) 
    # mfccs_features = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    # mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    y, sr = librosa.load(file, sr=16000)
    
    y = y[:192000]  #12 Secs for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])   

    # Define window size and hop length
    win_length = int(sr * 1) # 1 second window
    hop_length = int(win_length / 2) # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y)-win_length, hop_length):
        mfcc = librosa.feature.mfcc(y[i:i+win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

    
    # return mfccs_scaled_features

In [14]:
# mfcc_extracted_features=[]

# for index_num,row in tqdm(metadata.iterrows()):
#     file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#     final_class_labels=row["class"]
    
#     # MFCC Features Extraction
#     mfcc_data= mfcc_features_extractor_1sec(file_name)
#     mfcc_extracted_features.append([mfcc_data,final_class_labels])
#     # print(mfcc_extracted_features)

In [15]:
mfcc_data_by_fold = {}

# Iterate through the DataFrame and extract MFCC features
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]

    # MFCC Features Extraction
    mfcc_data = mfcc_features_extractor_1sec(file_name)
    
    # Store the MFCC data in the dictionary by fold number
    fold_num = row["fold"]
    if fold_num not in mfcc_data_by_fold:
        mfcc_data_by_fold[fold_num] = {
            "data": [],
            "labels": []
        }
    mfcc_data_by_fold[fold_num]["data"].append(mfcc_data)
    mfcc_data_by_fold[fold_num]["labels"].append(final_class_labels)

# Now, convert the data in mfcc_data_by_fold to NumPy arrays
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = np.array(data_dict["data"])
    data_dict["labels"] = np.array(data_dict["labels"])

0it [00:00, ?it/s]C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.00085449  0.00186157  0.00138855 ... -0.10037231 -0.09729004
 -0.05471802] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.4938507   0.44169617  0.36024475 ... -0.01057434  0.01789856
 -0.01347351] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.02241516  0.01522827  0.04211426 ...  0.03059387  0.01037598
 -0.02656555] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.04629517 -0.07

In [16]:
fold_num = 1  # Replace with the fold number you're interested in
data_shape = mfcc_data_by_fold[fold_num]["data"].shape
labels_shape = mfcc_data_by_fold[fold_num]["labels"].shape

print(f"Shape of data for fold {fold_num}: {data_shape}")
print(f"Shape of labels for fold {fold_num}: {labels_shape}")

Shape of data for fold 1: (6, 704, 40)
Shape of labels for fold 1: (6,)


In [17]:
mfcc_data_by_fold[3]["data"].shape

(6, 704, 40)

In [18]:
# Iterate through the dictionary and reshape the data arrays for all fold_num values
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = data_dict["data"].reshape(data_dict["data"].shape + (1,))
    

In [19]:
mfcc_data_by_fold[1]["data"].shape

(6, 704, 40, 1)

After Training Work, Deep Features 

In [20]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# List of inputs for each fold's MFCC data
input_data_list = [mfcc_data_by_fold[fold]["data"] for fold in range(1, 4)]  # Adjust the range as needed

# List to store intermediate layer outputs for each fold
intermediate_layer_outputs = []

# Iterate through the input data for each fold and predict the intermediate layer output
for input_data in input_data_list:
    intermediate_output = intermediate_layer_model.predict(input_data)
    intermediate_layer_outputs.append(intermediate_output)

# Now, intermediate_layer_outputs contains the intermediate layer outputs for each fold's input data


1/1 [==============================] - 0s 68ms/step


In [21]:
intermediate_layer_outputs = np.array(intermediate_layer_outputs)

In [22]:
intermediate_layer_outputs.shape

(3, 6, 1024)

In [23]:
intermediate_layer_outputs

array([[[-0.9017036 ,  0.0535625 ,  0.9949409 , ...,  0.9909545 ,
         -0.81006706, -0.99704915],
        [-0.1504857 ,  0.3799457 ,  0.9718634 , ...,  0.9170883 ,
         -0.983991  , -0.8841925 ],
        [-0.92282397, -0.99893904,  0.9992736 , ...,  0.9658053 ,
         -0.95049787, -0.96679765],
        [-0.8493272 ,  0.9999997 ,  0.9992713 , ...,  0.9986727 ,
         -0.9998277 , -0.9594157 ],
        [ 0.7855348 ,  0.97540474, -0.01935797, ...,  0.6780503 ,
         -0.3726188 ,  0.21572536],
        [-0.91612566,  0.9836924 ,  0.9846686 , ...,  0.99813724,
         -0.99933136, -0.9957763 ]],

       [[-0.9849227 , -1.        ,  0.9998739 , ...,  0.7557078 ,
         -0.05237596, -0.9999937 ],
        [-0.9998998 ,  0.02440293,  0.99999243, ...,  0.99292845,
         -0.9993669 , -1.        ],
        [-0.9996846 , -0.997938  ,  0.99993557, ...,  0.988009  ,
         -0.9962565 , -1.        ],
        [-0.99994975, -0.9702041 ,  1.        , ...,  0.9990437 ,
          0.99

In [24]:
mean_across_folds = np.mean(intermediate_layer_outputs, axis=0)

In [25]:
an_nas_mean = mean_across_folds

In [26]:
an_nas_mean.shape

(6, 1024)

In [27]:
an_nas_mean

array([[-0.66558325, -0.00306586,  0.9982691 , ...,  0.9014936 ,
        -0.6207734 , -0.9989464 ],
       [-0.52929413, -0.15615381,  0.9889409 , ...,  0.8604827 ,
        -0.98927337, -0.9257741 ],
       [-0.74063903, -0.33307514,  0.9862051 , ...,  0.9363939 ,
        -0.98193425, -0.9803849 ],
       [-0.8658716 ,  0.1453663 ,  0.99946094, ...,  0.99857974,
        -0.33301488, -0.98635745],
       [-0.3440249 ,  0.55542874,  0.6601674 , ...,  0.88822603,
        -0.12355361, -0.594744  ],
       [-0.9432078 ,  0.97835374,  0.99486464, ...,  0.96003133,
        -0.3334968 , -0.99813986]], dtype=float32)

In [28]:
# Specify the file path where you want to save the .npy file
file_path = 'an_nas_mean.npy'

# Save the array as a .npy file
np.save(file_path, an_nas_mean)

Testing of Single Audio

In [29]:
file_name = 'C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/nas_testing.wav'


mfcc_extracted_features_single=[]

mfcc_extracted_features_single= mfcc_features_extractor_1sec(file_name)

mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)

mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))



C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.00947571 -0.03855896 -0.02676392 ... -0.2828064  -0.21446228
 -0.15783691] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.04492188 -0.2520752  -0.13150024 ... -0.14852905 -0.1995697
 -0.33621216] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.12696838 -0.20048523 -0.35925293 ...  0.06340027  0.07987976
  0.09310913] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.43867493 -0.43997192 -0.41949463 

In [30]:
mfcc_extracted_features_single

array([[[[-2.3347307e+02],
         [ 7.7957184e+01],
         [-1.1075085e+01],
         ...,
         [ 2.4752839e+00],
         [ 6.5202942e+00],
         [ 9.2671859e-01]],

        [[-2.2442651e+02],
         [ 6.4169304e+01],
         [-1.5077600e+01],
         ...,
         [ 4.6533422e+00],
         [ 9.7718315e+00],
         [ 7.9807925e+00]],

        [[-2.3536192e+02],
         [ 5.9261177e+01],
         [-2.0262638e+01],
         ...,
         [ 5.6177855e+00],
         [ 8.3177071e+00],
         [ 9.4689560e+00]],

        ...,

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         [ 0.0000000e+00]],

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         [ 0.0000000e+00]],

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
 

In [31]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# Assuming you already have mfcc_extracted_features of shape (704, 40, 1)
# Replace this with your actual input data

# Predict the intermediate layer output for the input data
intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

# Now, intermediate_output contains the intermediate layer output for mfcc_extracted_features


1/1 [==============================] - 0s 102ms/step


In [32]:
intermediate_output

array([[-0.9972535 ,  0.953696  ,  0.9999988 , ...,  0.8960913 ,
         0.99883604, -0.9986993 ]], dtype=float32)

In [33]:
intermediate_output.shape

(1, 1024)

In [34]:
intermediate_layer_outputs

array([[[-0.9017036 ,  0.0535625 ,  0.9949409 , ...,  0.9909545 ,
         -0.81006706, -0.99704915],
        [-0.1504857 ,  0.3799457 ,  0.9718634 , ...,  0.9170883 ,
         -0.983991  , -0.8841925 ],
        [-0.92282397, -0.99893904,  0.9992736 , ...,  0.9658053 ,
         -0.95049787, -0.96679765],
        [-0.8493272 ,  0.9999997 ,  0.9992713 , ...,  0.9986727 ,
         -0.9998277 , -0.9594157 ],
        [ 0.7855348 ,  0.97540474, -0.01935797, ...,  0.6780503 ,
         -0.3726188 ,  0.21572536],
        [-0.91612566,  0.9836924 ,  0.9846686 , ...,  0.99813724,
         -0.99933136, -0.9957763 ]],

       [[-0.9849227 , -1.        ,  0.9998739 , ...,  0.7557078 ,
         -0.05237596, -0.9999937 ],
        [-0.9998998 ,  0.02440293,  0.99999243, ...,  0.99292845,
         -0.9993669 , -1.        ],
        [-0.9996846 , -0.997938  ,  0.99993557, ...,  0.988009  ,
         -0.9962565 , -1.        ],
        [-0.99994975, -0.9702041 ,  1.        , ...,  0.9990437 ,
          0.99

Euclidian Distance with Mean

In [35]:
import numpy as np

# Assuming you have al_fil_mean with shape (5, 1024) and intermediate_output with shape (1, 1024)
# Calculate the Euclidean distances between intermediate_output and each row in al_fil_mean
distances = np.linalg.norm(an_nas_mean - intermediate_output, axis=1)

# Find the index of the closest row in al_fil_mean
closest_index = np.argmin(distances)

# The value at closest_index in al_fil_mean is the closest to intermediate_output
closest_value = an_nas_mean[closest_index]

# Print the closest_index and closest_value
print(f"Index of closest value in an_nas_mean: {closest_index+1}")
print(f"Closest value in an_nas_mean: {closest_value}")

Index of closest value in an_nas_mean: 6
Closest value in an_nas_mean: [-0.9432078   0.97835374  0.99486464 ...  0.96003133 -0.3334968
 -0.99813986]


Euclidian Distance Without Mean

In [36]:
# # Reshape intermediate_output to match the shape of intermediate_layer_outputs
# intermediate_output_reshaped = intermediate_output.reshape((1, 1, 1024))

# # Initialize variables to keep track of the closest indices and distances
# closest_indices = None
# min_distances = None
# closest_set = None  # Initialize variable to track the closest set

# for i, intermediate_set in enumerate(intermediate_layer_outputs):
#     # Calculate the Euclidean distances between intermediate_output and intermediate_set
#     distances = np.linalg.norm(intermediate_set - intermediate_output_reshaped, axis=2)
    
#     # Find the index of the minimum distance
#     current_closest_index = np.argmin(distances)
    
#     # Update the closest index, distances, and closest set if this set is closer
#     if closest_indices is None or np.mean(distances) < np.mean(min_distances):
#         closest_indices = current_closest_index
#         min_distances = distances
#         closest_set = i

# # Print the closest index in intermediate_layer_outputs and which set it belongs to
# print(f"Closest index in intermediate_layer_outputs: {closest_indices}")
# if closest_set == 0:
#     print("It is closest to intermediate_layer_outputs[0]")
# elif closest_set == 1:
#     print("It is closest to intermediate_layer_outputs[1]")
# elif closest_set == 2:
#     print("It is closest to intermediate_layer_outputs[2]")